<a href="https://colab.research.google.com/github/ElUnrast/GermanWave2Vec/blob/main/Wave2Vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 0. Configuartion

In [ ]:
import os
import sys

device = 'cuda'
wait_for_v100 = False
ds_to_us = []
home = os.getenv("HOME")
use_gdrive = False
trained_model_directory = 'trained_model'
WANDB_API_KEY='6cec6a71175dec59b32a9067a0664dec76208f2a'
WANDB_NAME='GermanWaveToVec Training'
WANDB_ENTITY='wandb.ai/elunrast'
WANDB_PROJECT='GermanWaveToVec'
# Use Model from Checkpoint. 
# The Value must be a Sub-Directory of {trained_model_path} (Example: 'checkpoint-3600')
checkpoint = None

## ----- Training arguments  ----- ##
per_device_train_batch_size = 5      # TODO: aus freiem Grafikkarten Speicher ableiten (Graphikkarte mit 11GB)
per_device_eval_batch_size  = per_device_train_batch_size // 2
gradient_accumulation_steps = 2
logging_steps               = 100
max_steps                   = per_device_train_batch_size * logging_steps
max_trainingset_size        = max_steps * per_device_train_batch_size # TODO freiem Grafikkarten Speicher ableiten
max_steps                   = max_trainingset_size // per_device_train_batch_size # Schritte pro Epoche
warmup_steps                = max_steps // 4
save_steps                  = max_steps // 2
eval_steps                  = save_steps
max_sample_size             = 1200   # None -> no restriction
max_trainingset_size        = per_device_train_batch_size * max_steps   # None -> no restriction
fixed_training_set_size     = 100 * per_device_eval_batch_size          # None -> 20% der Originaldatensatz Größe 
num_train_epochs            = 3  # Anzahl Durchläufe für ein Dataset
max_rounds                  = 4  # Anzahl Trainingsrunden über alle used_datasets

learning_rate = per_device_train_batch_size * ((3e-4)/16)  # 3e-4/4 

## 1. Setup Directories

In [ ]:
local_nlp_base_dir = f'{home}/shared'
local_data_base_dir = f'{local_nlp_base_dir}/NLP-Data'
local_data_dir = f'{local_data_base_dir}/audio'
extern_nlp_base_dir = None
extern_data_base_dir = None

runs_on_colab = (home == '/root')
print( f'runs on colab: {runs_on_colab}')

if not runs_on_colab:
    os.environ['http_proxy'] = 'http://192.168.8.50:3128'
    os.environ['https_proxy'] = 'http://192.168.8.50:3128'
else:
    # to get access to the datasets we use gdrive
    use_gdrive = True
    # install packages
    !pip install datasets==1.4.1
    !pip install transformers==4.4.0
    !pip install jiwer
    !pip install torchaudio
    !pip install librosa
    # create local directories
    !mkdir $local_nlp_base_dir
    !mkdir $local_data_base_dir

if use_gdrive:
    gdrive_base = '/content/gdrive'    
    extern_nlp_base_dir = f'{gdrive_base}/MyDrive'
    extern_data_base_dir = f'{extern_nlp_base_dir}/NLP-Data'
    extern_data_dir = f'{extern_data_base_dir}/audio'

    if not os.path.isdir(gdrive_base):
        from google.colab import drive
        drive.mount(gdrive_base)

if not os.path.isdir(local_data_dir):
    !mkdir $local_data_dir

if extern_nlp_base_dir:
    model_dir = f'{extern_nlp_base_dir}/NLP-Models/GermanWave2Vec'
else:
    model_dir = f'{local_nlp_base_dir}/NLP-Models/GermanWave2Vec'

# Use the Model from this Directory (Base: .../NLP-Models/GermanWave2Vec/)
# None -> Start from 'facebook/wav2vec2-large-xlsr-53-german'
trained_model_path = f'{model_dir}/{trained_model_directory}'
git_views_dir = f'{local_nlp_base_dir}/gitviews'

if not os.path.isdir(git_views_dir):
    !mkdir $git_views_dir
    !cd $git_views_dir; git clone https://github.com/ElUnrast/GermanWave2Vec.git

runs on colab: True
     |████████████████████████████████| 194kB 16.3MB/s 
     |████████████████████████████████| 245kB 23.5MB/s 
     |████████████████████████████████| 112kB 25.3MB/s 
     |████████████████████████████████| 2.1MB 13.9MB/s 
     |████████████████████████████████| 901kB 50.2MB/s 
     |████████████████████████████████| 3.3MB 50.1MB/s 
     |████████████████████████████████| 51kB 6.8MB/s 
  Created wheel for python-Levenshtein: filename=python_Levenshtein-0.12.2-cp37-cp37m-linux_x86_64.whl size=149819 sha256=ee8fb6ee8bc6899d577dac00dc49258bca9f36c1d0e407d6fac436bb76df7e09
  Stored in directory: /root/.cache/pip/wheels/b3/26/73/4b48503bac73f01cf18e52cd250947049a7f339e940c5df8fc
Successfully built python-Levenshtein
     |████████████████████████████████| 1.9MB 10.5MB/s 
Mounted at /content/gdrive
Cloning into 'GermanWave2Vec'...
remote: Enumerating objects: 43, done.
remote: Counting objects: 100% (43/43), done.
remote: Compressing objects: 100% (29/29), done.
remote: 

In [ ]:
if runs_on_colab:
    git_view_path = f'{git_views_dir}/GermanWave2Vec'
    !cd $git_view_path; git fetch --all; git reset --hard origin/main

script_path = f'{git_views_dir}/GermanWave2Vec/python'
sys.path.insert(0, script_path)

Fetching origin
HEAD is now at 32f6bfb bugfix


## 2. Check Runtime Properties

In [ ]:
if 'cuda' == device:
    gpu_info = !nvidia-smi
    gpu_info = '\n'.join(gpu_info)

    if runs_on_colab:
        if wait_for_v100 and not gpu_info.find('V100') >= 0:
            print('The current GPU is not a V100')
            print('Since you want to wait for a V100 the current session is aborted')
            raise ValueError

        if gpu_info.find('failed') >= 0:
            print('For training, please use a VM with GPU!')
            raise ValueError
            
        from psutil import virtual_memory
        ram_gb = virtual_memory().total / 1e9
        print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

        if ram_gb < 20:
            print('To enable a high-RAM runtime, select the Runtime > "Change runtime type"')
            print('menu, and then select High-RAM in the Runtime shape dropdown. Then, ')
            print('re-execute this cell.')
        else:
            print('You are using a high-RAM runtime!')
            
    print(gpu_info)

Your runtime has 13.6 gigabytes of available RAM

To enable a high-RAM runtime, select the Runtime > "Change runtime type"
menu, and then select High-RAM in the Runtime shape dropdown. Then, 
re-execute this cell.
Wed May 12 09:37:47 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               | 

## 3 Install packages and do Imports

In [ ]:
from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

import torch
import torchaudio
from torch import nn
from torch.cuda.amp import autocast

from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor, Wav2Vec2CTCTokenizer, Wav2Vec2FeatureExtractor
from transformers import Trainer, TrainingArguments
from transformers.trainer_utils import get_last_checkpoint, is_main_process
from transformers.trainer_pt_utils import LengthGroupedSampler, DistributedLengthGroupedSampler

import json
import collections
import librosa
import numpy as np
import pandas as pd
import sklearn
import jiwer
from jiwer import wer
from datasets import load_metric
from tqdm.notebook import tqdm_notebook
from sklearn.model_selection import train_test_split

from SnippetDatasets import SnippetDatasets
from GermanSpeechToTextTranslater import GermanSpeechToTextTranslater

## 4 Initialize Helper Classes

In [ ]:
my_datasets = SnippetDatasets(
    runs_on_colab, 
    local_audio_base_dir=local_data_dir, 
    extern_audio_base_dir=extern_data_dir
)

In [ ]:
model_path = None

if checkpoint:
    model_path = f'{trained_model_path}/{checkpoint}'
elif os.path.isfile(f'{trained_model_path}/pytorch_model.bin'):
    model_path = trained_model_path

print(f'Initialize Model from Path: {model_path}')
translator = GermanSpeechToTextTranslater(model_name=model_path)

Initialize Model from Path: /content/gdrive/MyDrive/NLP-Models/GermanWave2Vec/trained_model
Using Model: /content/gdrive/MyDrive/NLP-Models/GermanWave2Vec/trained_model
Loading processor
Loading metric



Loading model


In [ ]:
def train():
  training_args = TrainingArguments(
      output_dir=trained_model_path,
      group_by_length=True,
      per_device_train_batch_size=per_device_train_batch_size,
      per_device_eval_batch_size=per_device_eval_batch_size, 
      gradient_accumulation_steps=gradient_accumulation_steps, 
      evaluation_strategy="steps",
      max_steps=max_steps,         
      fp16=True,
      save_steps=save_steps,     
      eval_steps=eval_steps,     
      logging_steps=logging_steps, 
      learning_rate=learning_rate,
      warmup_steps=warmup_steps,   
      save_total_limit=2
  )

  ds_to_train = []

  for ds in get_used_datasets():
      ds_to_train.append(ds)
      
  calc_wer_after_training = False  # Wegen OutOfMemory ausgeschaltet 

  for runde in range(max_rounds):
      print(f'Starting round {runde} of {max_rounds}')

      for ds_id in ds_to_train:
          if not my_datasets.has_translation_with_original(ds_id):
              ds_to_train.remove(ds_id)
              continue

          WANDB_NOTES=ds_id
          stop = False
          pandas_df = my_datasets.load_ds_content_translated_with_original(ds_id)
          print(f'Dataset - {ds_id} loaded with {pandas_df.shape[0]} Entries')
          mp3_dir = my_datasets.get_snippet_directory(ds_id)
          
          for epoche in range(num_train_epochs):
              print(f'Starting round {runde} of {max_rounds}, epoche {epoche} of {num_train_epochs}')
              print(f'Splitting Dataset {ds_id} with {pandas_df.shape[0]} Entries')
              train_pandas_ds, test_pandas_ds = translator.split_dataset(
                  pandas_df, 
                  max_trainingset_size=max_trainingset_size, 
                  max_sample_size=max_sample_size
              )
              
              if train_pandas_ds.shape[0] > pandas_df.shape[0] * 0.02:            
                  print(f'Creating Trainer for {ds_id}')
                  trainer = translator.get_trainer(
                      training_args, 
                      mp3_dir,             
                      train_pandas_ds,
                      test_pandas_ds,
                      use_grouped_legth_trainer=False
                  )
                  print(f'Training of Dataset: {ds_id}')
                  train_result = trainer.train()

                  print(f'Save Model')
                  trainer.save_model()
                  metrics = train_result.metrics
                  max_train_samples = train_pandas_ds.shape[0]
                  metrics["train_samples"] = min(max_train_samples, train_pandas_ds.shape[0])
                  trainer.log_metrics("train", metrics)
                  trainer.save_metrics("train", metrics)
                  trainer.save_state()
                  torch.cuda.empty_cache()

                  last_checkpoint = get_last_checkpoint(trained_model_path)
                  print( f'last Checkpoint: {last_checkpoint}')
                  
                  # print(f'Reload from last Checkpoint: {last_checkpoint}')
                  # translator.reload_from_checkpoint(last_checkpoint)
                  # {last_checkpoint}-
                  bad_translation_ds, truncated_ds, wer_result = translator.test(
                      ds_id, 
                      pandas_df, 
                      mp3_dir, 
                      diff_file_extension=f'{epoche}-{runde}', 
                      diff_calc_wer=calc_wer_after_training
                  )
                  
                  if (bad_translation_ds.shape[0] < (truncated_ds.shape[0] * 0.02)) or (wer_result < 0.02):
                      # mindestens 98% der Sätze wurde korrekt übersetzt. Überprüfung der Problemfälle ist angebracht.
                      # Es hat sich gezeigt, dass das Ergebnis wieder schlechter werden kann.
                      stop = True
              else:
                  stop = True

              if stop:
                  print('Early stopping!')
                  ds_to_train.remove(ds_id)
                  break

  print('Training finisched!')

## 4. Choose Dataset

In [ ]:
all_ds_ids = set()
all_ds_ids.update(list(my_datasets.local_datasets.keys()))
all_ds_ids.update(list(my_datasets.extern_datasets.keys()))

import difflib
from functools import partial

import ipywidgets as widgets
from IPython.display import display, Audio

ds_checkbox_items = []

for ds_id in sorted(all_ds_ids):
    ds_checkbox_items.append(widgets.Checkbox(
        value=ds_id in ds_to_us, 
        description=ds_id, 
        disabled=False, 
        indent=False
    ))

def get_used_datasets(checkbox_items=ds_checkbox_items):
    result = []

    for item in checkbox_items:
        if item.value:
            result.append(item.description)

    return result

print('Please choose Datasets to use')
ds_checkboxes_widget = widgets.Box(ds_checkbox_items)
display(ds_checkboxes_widget)

Please choose Datasets to use


Box(children=(Checkbox(value=False, description='HP1-FvM', indent=False), Checkbox(value=False, description='H…

In [ ]:
def on_train_action_button_clicked(b):
    print('Train')
    train()

def on_translate_action_button_clicked(b):
    print('Translate')
    for ds_id in get_used_datasets():
        translator.translate_and_extend_dataset_from_directory(my_datasets, ds_id)    

class InvisibleAudio(Audio):
    def _repr_html_(self):
        audio = super()._repr_html_()
        audio = audio.replace('<audio', f'<audio onended="this.parentNode.removeChild(this)"')
        return f'<div style="display:none">{audio}</div>'

def show_diff(text, n_text):
    """
    http://stackoverflow.com/a/788780
    Unify operations between two compared strings seqm is a difflib.
    SequenceMatcher instance whose a & b are strings
    """
    seqm = difflib.SequenceMatcher(None, text, n_text)
    output= []
    for opcode, a0, a1, b0, b1 in seqm.get_opcodes():
        if opcode == 'equal':
            output.append(seqm.a[a0:a1])
        elif opcode == 'insert':
            output.append("<font color=red>^" + seqm.b[b0:b1] + "</font>")
        elif opcode == 'delete':
            output.append("<font color=blue>^" + seqm.a[a0:a1] + "</font>")
        elif opcode == 'replace':
            # seqm.a[a0:a1] -> seqm.b[b0:b1]
            output.append("<font color=green>^" + seqm.b[b0:b1] + "</font>")
        else:
            raise RuntimeError("unexpected opcode")
    return ''.join(output)
    
def create_diff_row(audio_file, original_text, translated_text ):
    diff_widget = widgets.HTML(value=show_diff(original_text, translated_text))
    button_widget = widgets.Button(description='Play')

    def on_play_button_clicked(f, b):
        display(InvisibleAudio(url=audio_file, autoplay=True))

    button_widget.on_click(partial(on_play_button_clicked, 1))
    return widgets.HBox([button_widget, diff_widget])

def create_diff_content(ds_id):
    ds = my_datasets.load_ds_content_translated_with_original(ds_id)
    snipped_directory = my_datasets.get_snippet_directory(ds_id)
    print(f'Use Snipped Directory: {snipped_directory}')
    translation_row = 'Translated1' if 'Translated1' in ds.columns else 'Translated0'
    ds = ds[ds['OriginalText'] != ds[translation_row]]
    rows = []
    max_len = min(20, len(ds))

    for idx in tqdm_notebook(range(max_len)):
        rows.append(create_diff_row(
            f'{snipped_directory}/{ds.iloc[idx]["Datei"]}', 
            ds.iloc[idx]['OriginalText'], 
            ds.iloc[idx][translation_row]
        ))

    return widgets.VBox(rows)

def on_validate_action_button_clicked(b):
    print('Validate')
    tab = widgets.Tab()
    tab_titles = []
    tab_children = []

    for ds_id in get_used_datasets():
        tab_titles.append(ds_id)
        tab_children.append(create_diff_content(ds_id))

    tab.children = tab_children
  
    for idx, name in enumerate(tab_titles):
        tab.set_title(idx, name)
  
    display(tab)


train_action_button = widgets.Button(description='Train')
train_action_button.on_click(on_train_action_button_clicked)
translate_action_button = widgets.Button(description='Translate')
translate_action_button.on_click(on_translate_action_button_clicked)
validate_action_button = widgets.Button(description='Validate')
validate_action_button.on_click(on_validate_action_button_clicked)

action_buttons = widgets.HBox([
    train_action_button, 
    translate_action_button, 
    validate_action_button
])
display(action_buttons)

Validate
Loading Dataset: HP1-RB - content-translated-with_original.csv
Download and extract /content/gdrive/MyDrive/NLP-Data/audio/HP1-RB.zip from gdrive
Pruning Dataset HP1-RB with 10555 Entries
 - 10555 Entries left after Length Cut (min=31, max=4000)
 - 5162 Entries left after Action Cut
Dataset was truncated from 10555 to 5162 Entries. Saving Backup.
Use Snipped Directory: /root/shared/NLP-Data/audio/HP1-RB


## 5. Translate if nessecary only on local Notebook

In [ ]:
# ds_id = 'HP1-FvM'
# ds = my_datasets.load_ds_content_translated_with_original(ds_id)
# ds = ds[ds['OriginalText'] != ds['Translated0']]

## 6. Train

In [ ]:
# for i in range(len(ds)):
#   print(ds.iloc[i]['Translated0'])

In [ ]:
# translator.reload_from_checkpoint(last_checkpoint)
# predictions, _ = translator.translate_dataset(mp3_dir, test_dataset)
# labels = []

# for datei in test_dataset.paths:
#     labels.extend( pandas_df[pandas_df['Datei'] == datei]['OriginalText'].tolist())
    
# print(wer(labels, predictions))

In [ ]:
# labels = []

# for datei in test_dataset.paths:
#     labels.extend( pandas_df[pandas_df['Datei'] == datei]['OriginalText'].tolist())

# print(wer(labels, predictions))

In [ ]:
# orig_file = open(f'{model_dir}/orig-{use_datasets[0]}.txt', 'w')
# cp = os.path.basename(last_checkpoint)
# translated_file = open(f'{model_dir}/orig-{use_datasets[0]}-{cp}.txt', 'w')

# for file_name, label, prediction in zip(test_dataset.paths, labels, predictions):
#         orig_file.write(f'{file_name}, {label}\n')    
#         translated_file.write(f'{file_name}, {prediction}\n')    

# orig_file.close()
# translated_file.close()

In [ ]:
# translator.my_processor.tokenizer

In [ ]:
# translator.my_processor.tokenizer.get_vocab()

In [ ]:
# big_df = pandas_df[pandas_df.Length > 1200]
# print(big_df.shape[0])

In [ ]:
# big_df

In [ ]:
# small_df = pandas_df[pandas_df.OriginalText.str.len() < 10]

In [ ]:
# small_df

In [ ]:
#ds = my_datasets.load_ds_content_translated_with_original(use_datasets[0])
#del ds['Translation0']
#ds.to_csv(f'{my_datasets.get_snippet_directory(use_datasets[0])}/content-translated-with_original.csv', sep=';')

In [ ]:
# truncated_ds = pandas_df[
#     ((pandas_df.Action == 'train') | (pandas_df.Action == 'translate')) & (pandas_df.Length < 1200)
# ]
# truncated_ds

In [ ]:
# id_or_directory = use_datasets[0]
# mp3_dir = my_datasets._get_directory(id_or_directory)
# pandas_df = pd.read_csv(f'{mp3_dir}/content-translated-with_original.csv', sep=';')
# pandas_df[~pandas_df.Action.str.startswith('exclude')]